# Performing needed imports and boilder plate code.

In [14]:
import random
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import torch
from torch.utils.data import DataLoader

# set this variable to a number to be used as the random seed
# or to None if you don't want to set a random seed
seed = 1234

if seed is not None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

# Copied over data cleaning steps from our data cleaning notebook.

In [15]:
education_data = pd.read_csv('students_clean.csv')


education_data.drop('Parent_Education_Level', axis=1, inplace=True) 

education_data['Gender'] = education_data['Gender'].replace({'Male': 1, 'Female': 0}).astype(int)
education_data['Internet_Access_at_Home'] = education_data['Internet_Access_at_Home'].replace({'Yes': 1, 'No': 0}).astype(int)
education_data['Extracurricular_Activities'] = education_data['Extracurricular_Activities'].replace({'Yes': 1, 'No': 0}).astype(int)


# Low = 1, Medium = 2, High = 3
mapper = {'low': 1, 'medium': 2, 'high': 3}

education_data['Family_Income_Level'] = (
    education_data['Family_Income_Level']
      .astype(str)                  # works even if the value is already 1/2/3 or NaN
      .str.strip().str.lower()
      .map(mapper)                  # returns NaN where no mapping found
      .fillna(education_data['Family_Income_Level'])  # keepin the original numeric/blank entries
      .astype('Int64')              #  nullable integer dtype
)

labels = open('departments.txt').read().splitlines()
department_mapping = {name: index for index, name in enumerate(labels)}
department_indices = education_data['Department'].map(department_mapping)
education_data.insert(3, 'department index', department_indices)

mapper = {'A': 1, 'B': 1, 'C': 1, 'D':0,'F':0}

education_data['Grade'] = (
    education_data['Grade']
      .astype(str)              # convert everything to string
      .str.strip().str.upper()  # remove spaces and standardize to uppercase
      .map(mapper)              # map letters to numbers
)
education_data = education_data.drop(columns='Department')
education_data.head()

/tmp/ipykernel_709327/2188593247.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  education_data['Gender'] = education_data['Gender'].replace({'Male': 1, 'Female': 0}).astype(int)
/tmp/ipykernel_709327/2188593247.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  education_data['Internet_Access_at_Home'] = education_data['Internet_Access_at_Home'].replace({'Yes': 1, 'No': 0}).astype(int)
/tmp/ipykernel_709327/2188593247.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version

,Gender,Age,department index,Attendance (%),Midterm_Score,Final_Score,Assignments_Avg,Quizzes_Avg,Participation_Score,Projects_Score,Total_Score,Grade,Study_Hours_per_Week,Extracurricular_Activities,Internet_Access_at_Home,Family_Income_Level,Stress_Level (1-10),Sleep_Hours_per_Night
0,0,22,0,97.36,40.61,59.61,73.69,53.17,73.4,62.84,59.8865,0,10.3,1,0,2,1,5.9
1,1,18,1,97.71,57.27,74.00,74.23,98.23,88.0,98.23,81.9170,1,27.1,0,0,1,4,4.3
2,1,24,2,99.52,41.84,63.85,85.85,50.00,4.7,91.22,67.7170,0,12.4,1,0,1,9,6.1
3,0,24,2,90.38,45.65,44.44,68.10,66.27,4.2,55.48,51.6535,0,25.5,0,1,1,8,4.9
4,0,23,3,59.41,53.13,61.77,67.66,83.98,64.3,87.43,71.4030,1,13.3,1,0,2,6,4.5


# Now defining our data loader and perceptron

In [16]:
from torch.utils.data import Dataset
import torch

class MyDataset(Dataset):
    def __init__(self, df, feature_cols, target_col):
        self.df = df
        self.feature_cols = feature_cols
        self.target_col = target_col

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        x = torch.tensor(row[self.feature_cols].to_numpy(dtype=np.float32), dtype=torch.float32)
        y = torch.tensor(row[self.target_col], dtype=torch.long)  # long for classification
        return x, y

In [17]:
def train_perceptron(train_dl, n_features, pos_class):
    # First initialize the model.
    w = np.zeros(n_features)
    b = 0
    n_errors = 0
    weight_steps = []
    total_pos_in_train = 0

    # Adding this in for debug purposes to track the changes to the weight vectors on each
    # round.
    
    # Average perceptron features
    totalW = np.zeros(n_features)
    totalB = 0;
    updateCount = 0;
    
    # Now loop through each batch.
    for batch_idx, (x, y) in tqdm(enumerate(train_dl), total=len(train_dl),):
        
        x_curr_np = x.numpy()
        y_curr_np = y.numpy()

        total_pos_in_train += (y_curr_np == 1).sum(axis=0)
        

        # Now perform the training/classification loop.
        scores = x_curr_np @ w + b
       
        
        y_pred = (scores > 0).astype(int)


        # Now we vectorize the update to make this more efficient.
        pred_error = y_curr_np - y_pred
        n_errors += np.sum(np.abs(pred_error) != 0) # If the pred error is zero then it is correct.

        # First append the previous weights to weight steps which will be used for debuging puprposes.
        weight_steps.append((pred_error[:,None]*x_curr_np).sum(axis=0).copy())
        
        w += (pred_error[:,None]*x_curr_np).sum(axis=0) # Re-shape pred errors to update and only add
                                                        # inccorect preds, axis=0 for rows.
        b += pred_error.sum()

        # Now print out the weights and bias updates every update if we are in debug mode.
        

    # Now once we are done training the result is the weights and biases.
    return (w,b,n_errors,weight_steps.copy(),total_pos_in_train) # I am just copying to avoid weird cases due to mutability of list.

# Create the training and testing partitions.

In [18]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(education_data, train_size=0.9,random_state=seed)
train_df,dev_df = train_test_split(train_df, train_size=0.8,random_state=seed)

train_df.reset_index(inplace=True,drop=True)
dev_df.reset_index(inplace=True,drop=True)
test_df.reset_index(inplace=True,drop=True)


print(f'train rows: {len(train_df.index):,}')
print(f'dev rows: {len(dev_df.index):,}')
print(f'test rows: {len(test_df.index):,}')

train rows: 3,600
dev rows: 900
test rows: 500


In [19]:
# Check pass/fail distribution in train and dev datasets
print("="*70)
print("Pass/Fail Distribution in Datasets")
print("="*70)

# Train set distribution
train_pass = (train_df['Grade'] == 1).sum()
train_fail = (train_df['Grade'] == 0).sum()
train_total = len(train_df)

print(f"\nTrain Set:")
print(f"  Pass (1): {train_pass} ({train_pass/train_total*100:.2f}%)")
print(f"  Fail (0): {train_fail} ({train_fail/train_total*100:.2f}%)")
print(f"  Total: {train_total}")

# Dev set distribution
dev_pass = (dev_df['Grade'] == 1).sum()
dev_fail = (dev_df['Grade'] == 0).sum()
dev_total = len(dev_df)

print(f"\nDev Set:")
print(f"  Pass (1): {dev_pass} ({dev_pass/dev_total*100:.2f}%)")
print(f"  Fail (0): {dev_fail} ({dev_fail/dev_total*100:.2f}%)")
print(f"  Total: {dev_total}")

# Test set distribution
test_pass = (test_df['Grade'] == 1).sum()
test_fail = (test_df['Grade'] == 0).sum()
test_total = len(test_df)

print(f"\nTest Set:")
print(f"  Pass (1): {test_pass} ({test_pass/test_total*100:.2f}%)")
print(f"  Fail (0): {test_fail} ({test_fail/test_total*100:.2f}%)")
print(f"  Total: {test_total}")


Pass/Fail Distribution in Datasets

Train Set:
  Pass (1): 2105 (58.47%)
  Fail (0): 1495 (41.53%)
  Total: 3600

Dev Set:
  Pass (1): 555 (61.67%)
  Fail (0): 345 (38.33%)
  Total: 900

Test Set:
  Pass (1): 301 (60.20%)
  Fail (0): 199 (39.80%)
  Total: 500


# Shave Columns

In [20]:
def get_features(data, column_name, exclude_cols, n):

    data = data.drop(columns=exclude_cols)

    corr_df = data.corr(method='pearson')[[column_name]]
    corr_df = corr_df.drop(index=[column_name])
    corr_df['Grade_Corr_Abs'] = corr_df[column_name].apply(lambda x: np.abs(x))
    corr_df = corr_df.sort_values(by='Grade_Corr_Abs',ascending=False)
    corr_df = corr_df[corr_df['Grade_Corr_Abs'] >= 0.1]
    top_features = corr_df.head(n).index.tolist()
    return top_features

# Before Midterm

In [21]:
# Averaged Perceptron: Train 10 perceptrons and average their weights
n_perceptrons = 10
weight_vecs = []
bias_vecs = []
features_lst = get_features(education_data, 'Grade', ['Total_Score','Midterm_Score','Final_Score','Projects_Score'], 10)
num_feat = len(features_lst)
batch_size = 64
shuffle = True
for i in range(n_perceptrons):
    train_ds = MyDataset(train_df, features_lst, 'Grade')
    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=shuffle)
    
    w_curr, b_curr, error_curr, weight_hist_curr, tot_train_pos_curr = train_perceptron(train_dl, num_feat, pos_class=1)
    
    weight_vecs.append(w_curr)
    bias_vecs.append(b_curr)

# Average the weights and biases
w_avg = np.mean(weight_vecs, axis=0)
b_avg = np.mean(bias_vecs, axis=0)

print(f"-------------------Averaged Perceptron (10 perceptrons)------------------------------\n")
print(f"Averaged weight vector shape: {w_avg}")
print(f"Averaged bias value: {b_avg:.4f}\n")

# Test on dev.
X_dev_a = dev_df[features_lst].to_numpy()
dev_y_true = dev_df['Grade'].to_numpy()
dev_y_pred = ((X_dev_a @ w_avg + b_avg) > 0).astype(int)
n_correct_dev = (dev_y_true==dev_y_pred).sum(axis=0)

print(f"The number of correct preds was {n_correct_dev} for acc of {(n_correct_dev/dev_y_true.shape[0])*100}%")
print(f"The number of pos preds was {(dev_y_pred==1).sum(axis=0)} and neg num was {(dev_y_pred==0).sum(axis=0)}")

# Additional detailed metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

accuracy = accuracy_score(dev_y_true, dev_y_pred)
precision = precision_score(dev_y_true, dev_y_pred, zero_division=0)
recall = recall_score(dev_y_true, dev_y_pred, zero_division=0)
f1 = f1_score(dev_y_true, dev_y_pred, zero_division=0)
cm = confusion_matrix(dev_y_true, dev_y_pred)

print(f"\n{'='*70}")
print("Detailed Evaluation Metrics:")
print(f"{'='*70}")
print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

print(f"\nConfusion Matrix:")
print(f"                Predicted")
print(f"              Fail    Pass")
print(f"Actual Fail   {cm[0,0]:4d}   {cm[0,1]:4d}")
print(f"       Pass   {cm[1,0]:4d}   {cm[1,1]:4d}")

print(f"\nPredictions breakdown:")
print(f"  Predicted Fail (0): {(dev_y_pred==0).sum()}")
print(f"  Predicted Pass (1): {(dev_y_pred==1).sum()}")
print(f"  Actual Fail (0): {(dev_y_true==0).sum()}")
print(f"  Actual Pass (1): {(dev_y_true==1).sum()}")


  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

...

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

-------------------Averaged Perceptron (10 perceptrons)------------------------------

Averaged weight vector shape: [1278.50904045 -139.71191216 1375.66995692]
Averaged bias value: -66.4000

The number of correct preds was 555 for acc of 61.66666666666667%
The number of pos preds was 900 and neg num was 0

Detailed Evaluation Metrics:
Accuracy: 61.67%
Precision: 0.6167
Recall: 1.0000
F1-Score: 0.7629

Confusion Matrix:
                Predicted
              Fail    Pass
Actual Fail      0    345
       Pass      0    555

Predictions breakdown:
  Predicted Fail (0): 0
  Predicted Pass (1): 900
  Actual Fail (0): 345
  Actual Pass (1): 555


# After Midterm and before Final

In [22]:
# Averaged Perceptron: Train 10 perceptrons and average their weights
n_perceptrons = 10
weight_vecs = []
bias_vecs = []
features_lst = features_lst = get_features(education_data, 'Grade', ['Total_Score','Final_Score','Projects_Score'], 10)

num_feat = len(features_lst)

for i in range(n_perceptrons):
    train_ds = MyDataset(train_df, features_lst, 'Grade')
    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=shuffle)
    
    w_curr, b_curr, error_curr, weight_hist_curr, tot_train_pos_curr = train_perceptron(train_dl, num_feat, pos_class=1)
    
    weight_vecs.append(w_curr)
    bias_vecs.append(b_curr)

# Average the weights and biases
w_avg = np.mean(weight_vecs, axis=0)
b_avg = np.mean(bias_vecs, axis=0)

print(f"-------------------Averaged Perceptron (10 perceptrons)------------------------------\n")
print(f"Averaged weight vector shape: {w_avg}")
print(f"Averaged bias value: {b_avg:.4f}\n")

# Test on dev.
X_dev_a = dev_df[features_lst].to_numpy()
dev_y_true = dev_df['Grade'].to_numpy()
dev_y_pred = ((X_dev_a @ w_avg + b_avg) > 0).astype(int)
n_correct_dev = (dev_y_true==dev_y_pred).sum(axis=0)

print(f"The number of correct preds was {n_correct_dev} for acc of {(n_correct_dev/dev_y_true.shape[0])*100}%")
print(f"The number of pos preds was {(dev_y_pred==1).sum(axis=0)} and neg num was {(dev_y_pred==0).sum(axis=0)}")

# Additional detailed metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

accuracy = accuracy_score(dev_y_true, dev_y_pred)
precision = precision_score(dev_y_true, dev_y_pred, zero_division=0)
recall = recall_score(dev_y_true, dev_y_pred, zero_division=0)
f1 = f1_score(dev_y_true, dev_y_pred, zero_division=0)
cm = confusion_matrix(dev_y_true, dev_y_pred)

print(f"\n{'='*70}")
print("Detailed Evaluation Metrics:")
print(f"{'='*70}")
print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

print(f"\nConfusion Matrix:")
print(f"                Predicted")
print(f"              Fail    Pass")
print(f"Actual Fail   {cm[0,0]:4d}   {cm[0,1]:4d}")
print(f"       Pass   {cm[1,0]:4d}   {cm[1,1]:4d}")

print(f"\nPredictions breakdown:")
print(f"  Predicted Fail (0): {(dev_y_pred==0).sum()}")
print(f"  Predicted Pass (1): {(dev_y_pred==1).sum()}")
print(f"  Actual Fail (0): {(dev_y_true==0).sum()}")
print(f"  Actual Pass (1): {(dev_y_true==1).sum()}")

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

...

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

-------------------Averaged Perceptron (10 perceptrons)------------------------------

Averaged weight vector shape: [  393.32006416  1149.93708382 -1437.70390778  1290.33992902]
Averaged bias value: -85.1000

The number of correct preds was 575 for acc of 63.888888888888886%
The number of pos preds was 824 and neg num was 76

Detailed Evaluation Metrics:
Accuracy: 63.89%
Precision: 0.6396
Recall: 0.9495
F1-Score: 0.7643

Confusion Matrix:
                Predicted
              Fail    Pass
Actual Fail     48    297
       Pass     28    527

Predictions breakdown:
  Predicted Fail (0): 76
  Predicted Pass (1): 824
  Actual Fail (0): 345
  Actual Pass (1): 555


# After Final

In [23]:
# Averaged Perceptron: Train 10 perceptrons and average their weights
n_perceptrons = 10
weight_vecs = []
bias_vecs = []
features_lst = features_lst = get_features(education_data, 'Grade', ['Total_Score'], 10)

num_feat = len(features_lst)

for i in range(n_perceptrons):
    train_ds = MyDataset(train_df, features_lst, 'Grade')
    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=shuffle)
    
    w_curr, b_curr, error_curr, weight_hist_curr, tot_train_pos_curr = train_perceptron(train_dl, num_feat, pos_class=1)
    
    weight_vecs.append(w_curr)
    bias_vecs.append(b_curr)

# Average the weights and biases
w_avg = np.mean(weight_vecs, axis=0)
b_avg = np.mean(bias_vecs, axis=0)

print(f"-------------------Averaged Perceptron (10 perceptrons)------------------------------\n")
print(f"Averaged weight vector shape: {w_avg}")
print(f"Averaged bias value: {b_avg:.4f}\n")

# Test on dev.
X_dev_a = dev_df[features_lst].to_numpy()
dev_y_true = dev_df['Grade'].to_numpy()
dev_y_pred = ((X_dev_a @ w_avg + b_avg) > 0).astype(int)
n_correct_dev = (dev_y_true==dev_y_pred).sum(axis=0)

print(f"The number of correct preds was {n_correct_dev} for acc of {(n_correct_dev/dev_y_true.shape[0])*100}%")
print(f"The number of pos preds was {(dev_y_pred==1).sum(axis=0)} and neg num was {(dev_y_pred==0).sum(axis=0)}")

# Additional detailed metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

accuracy = accuracy_score(dev_y_true, dev_y_pred)
precision = precision_score(dev_y_true, dev_y_pred, zero_division=0)
recall = recall_score(dev_y_true, dev_y_pred, zero_division=0)
f1 = f1_score(dev_y_true, dev_y_pred, zero_division=0)
cm = confusion_matrix(dev_y_true, dev_y_pred)

print(f"\n{'='*70}")
print("Detailed Evaluation Metrics:")
print(f"{'='*70}")
print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

print(f"\nConfusion Matrix:")
print(f"                Predicted")
print(f"              Fail    Pass")
print(f"Actual Fail   {cm[0,0]:4d}   {cm[0,1]:4d}")
print(f"       Pass   {cm[1,0]:4d}   {cm[1,1]:4d}")

print(f"\nPredictions breakdown:")
print(f"  Predicted Fail (0): {(dev_y_pred==0).sum()}")
print(f"  Predicted Pass (1): {(dev_y_pred==1).sum()}")
print(f"  Actual Fail (0): {(dev_y_true==0).sum()}")
print(f"  Actual Pass (1): {(dev_y_true==1).sum()}")

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

...

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

-------------------Averaged Perceptron (10 perceptrons)------------------------------

Averaged weight vector shape: [ 5089.84199448  3209.22502289 -1626.95300293  -294.45893288
 -3480.15794106   993.59990144]
Averaged bias value: -115.7000

The number of correct preds was 564 for acc of 62.66666666666667%
The number of pos preds was 891 and neg num was 9

Detailed Evaluation Metrics:
Accuracy: 62.67%
Precision: 0.6229
Recall: 1.0000
F1-Score: 0.7676

Confusion Matrix:
                Predicted
              Fail    Pass
Actual Fail      9    336
       Pass      0    555

Predictions breakdown:
  Predicted Fail (0): 9
  Predicted Pass (1): 891
  Actual Fail (0): 345
  Actual Pass (1): 555


In [24]:
# Find False Positives (FP): Predicted Pass (1) but actual Fail (0)
false_pos_indices = np.where((dev_y_pred == 1) & (dev_y_true == 0))[0]
false_pos_examples = dev_df.iloc[false_pos_indices]

# Find False Negatives (FN): Predicted Fail (0) but actual Pass (1)
false_neg_indices = np.where((dev_y_pred == 0) & (dev_y_true == 1))[0]
false_neg_examples = dev_df.iloc[false_neg_indices]

# Get top 15 false positives and top 10 false negatives
false_pos_examples_top_15 = false_pos_examples.head(15)
false_neg_examples_top_10 = false_neg_examples.head(10)

# Print out the examples
print("15 Examples of False Positives (Predicted Pass, Actual Fail):")
print(false_pos_examples_top_15[['Attendance (%)', 'Extracurricular_Activities', 'Midterm_Score', 'Final_Score', 
                                  'Assignments_Avg', 'Quizzes_Avg', 'Participation_Score', 'Projects_Score', 
                                  'Stress_Level (1-10)', 'Sleep_Hours_per_Night', 'Grade']])

print("\n10 Examples of False Negatives (Predicted Fail, Actual Pass):")
print(false_neg_examples_top_10[['Attendance (%)', 'Extracurricular_Activities', 'Midterm_Score', 'Final_Score', 
                                  'Assignments_Avg', 'Quizzes_Avg', 'Participation_Score', 'Projects_Score', 
                                  'Stress_Level (1-10)', 'Sleep_Hours_per_Night', 'Grade']])


15 Examples of False Positives (Predicted Pass, Actual Fail):
    Attendance (%)  Extracurricular_Activities  Midterm_Score  Final_Score  \
1            92.68                           1          60.33        70.99   
8            70.32                           1          41.14        83.66   
16           68.04                           1          57.49        42.38   
19           52.96                           0          78.38        55.22   
20           68.14                           1          95.82        56.38   
22           72.90                           1          97.19        46.40   
23           87.30                           0          75.40        49.35   
27           96.62                           0          79.96        50.66   
31           66.87                           0          72.25        88.65   
32           68.47                           1          57.07        60.64   
35           88.60                           1          51.46        43.25   
36